<a href="https://colab.research.google.com/github/tomginsberg/stimdx/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/tomginsberg/stimdx.git qldpc

# stimdx

This notebook demonstrates the `stimdx` dynamic circuit framework. It allows mixing static Stim blocks with Python-controlled branching and looping.

## Setup

In [ ]:
import stim
from stimdx import Circuit, LastMeas
from stimdx import context as ctx
from rich.panel import Panel
from rich import print


def print_circuit(circ):
    print(Panel(f"{circ}"))


def print_samples(samples):
    print(["".join(map(lambda x: "1" if x else "0", s)) for s in samples])

## Example 0: Basic Control Flow

- Prepare 2 random bits
- Flip bit one if parity is odd

In [ ]:
circ = Circuit("H 0 1\nM 0 1")
# ctx is a proxy object that allows building expressions
# from the measurement record using logical operators
# ctx.rec is a symbol for the measurement record
circ.conditional(body="X 0", cond=ctx.rec(-1) ^ ctx.rec(-2))
circ.block("M 0 1")
print_circuit(circ)


sampler = circ.compile_sampler(seed=0)
samples = sampler.sample(shots=4)
print_samples(samples)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ StimBlock:                                                                                                      │
│   H 0 1                                                                                                         │
│   M 0 1                                                                                                         │
│ If (rec(-1) ^ rec(-2)):                                                                                         │
│   StimBlock:                                                                                                    │
│     X 0                                                                                                         │
│ StimBlock:                                                                                                      │
│   M 0 1                                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

['1000', '1111', '0111', '1000']

In [ ]:
circ = Circuit("H 0 1\nM 0 1")
# one level lower, we can use the runtime context object directly
circ.conditional(body="X 0", cond=lambda x: (x.meas_record[-1] ^ x.meas_record[-2]))
circ.block("M 0 1")
print_circuit(circ)


sampler = circ.compile_sampler(seed=0)
samples = sampler.sample(shots=4)
print_samples(samples)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ StimBlock:                                                                                                      │
│   H 0 1                                                                                                         │
│   M 0 1                                                                                                         │
│ If <function <lambda> at 0x75d02068cb80>:                                                                       │
│   StimBlock:                                                                                                    │
│     X 0                                                                                                         │
│ StimBlock:                                                                                                      │
│   M 0 1                                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

['1000', '1111', '0111', '1000']

Instead we could non destructively measure the parity onto a third qubit

In [ ]:
circ = Circuit("H 0 1\nCX 0 2 1 2\nM 2")
circ.conditional(body="X 0", cond=ctx.rec(-1))
circ.block("M 0 1")
print_circuit(circ)


sampler = circ.compile_sampler(seed=0)
samples = sampler.sample(shots=4)
print_samples(samples)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ StimBlock:                                                                                                      │
│   H 0 1                                                                                                         │
│   CX 0 2 1 2                                                                                                    │
│   M 2                                                                                                           │
│ If rec(-1):                                                                                                     │
│   StimBlock:                                                                                                    │
│     X 0                                                                                                         │
│ StimBlock:                                                                                                      │
│   M 0 1                                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

['100', '111', '011', '100']

## Example 1: Repeat-Until-Success (RUS)

This circuit attempts an operation that has a probabilistic success. It repeats until the desired outcome is measured.

Scenario: We want to prepare a '0' state, but our noise channel flips it to '1' with 50% probability. We check and retry if we get '1'.

In [ ]:
def build_rus_circuit():
    circ = Circuit()

    # Define the body of the loop
    # In a real RUS, this would be a probabilistic gate sequence.
    # Here we simulate a coin flip with H + Measure.
    trial_step_circ = Circuit()

    trial_block = stim.Circuit()
    trial_block.append("R", 0)  # Reset to clean state
    trial_block.append("H", 0)
    trial_block.append("M", 0)
    trial_step_circ.block(trial_block)

    # Add the loop to the main circuit
    # We want to STOP if we measure 0. So we CONTINUE while measurement is 1 (True).
    circ.do_while(body=trial_step_circ, cond=ctx.rec(-1))
    # Could also use this convenience for indexing directly into the last block
    # cond=lambda ctx: ctx.last_block_meas[0]

    return circ


rus_circ = build_rus_circuit()
print_circuit(rus_circ)
sampler = rus_circ.compile_sampler(seed=42)
samples = sampler.sample(shots=5)

print_samples(samples)
# Note: The output length varies! Each '1' is a failed attempt, ending in '0'.

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Do:                                                                                                             │
│   StimBlock:                                                                                                    │
│     R 0                                                                                                         │
│     H 0                                                                                                         │
│     M 0                                                                                                         │
│ While rec(-1)                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

['1111111110', '10', '11110', '111111110', '110']

## Example 2: Fault-Tolerant Steane Code Prep

This example reproduces the logic of a Fault-Tolerant Steane code preparation with post-selection.
We implement it as a Repeat-Until-Success loop, guaranteeing a valid state preparation.

In [ ]:
from qldpc.decoders import LookupDecoder
from qldpc.circuits import DepolarizingNoiseModel
from galois import GF
from qldpc.codes import SteaneCode

stean_parity = SteaneCode().matrix_z

ld = LookupDecoder(stean_parity, max_weight=3)


# 1. Define the Steane Code Preparation Block
def build_steane_block(noisy=True):
    c = stim.Circuit()

    # Data qubits: 0-6. Ancilla: 7
    data_qubits = list(range(7))
    ancilla = 7

    # Reset all
    c.append("R", data_qubits + [ancilla])

    # Non-FT Zero Prep
    plus_ids = [0, 4, 6]
    c.append("H", plus_ids)

    cx_pairs = [(0, 1), (4, 5), (6, 3), (6, 5), (4, 2), (0, 3), (4, 1), (3, 2)]
    for ctrl, targ in cx_pairs:
        c.append("CNOT", [ctrl, targ])

    # FT Verification (Flag check)
    # Verifies the stabilizer weights using an ancilla
    flags = [1, 3, 5]
    for f in flags:
        c.append("CNOT", [f, ancilla])

    # Measure ancilla to check for errors
    c.append("M", [ancilla])

    if noisy:
        nm = DepolarizingNoiseModel(p=1e-3)  # 1e-3 as in guppy example
        return nm.noisy_circuit(c)
    return c


def calculate_parity(bit_list):
    # Parity of the 7 data bits (last 7 bits of the sample)
    # The sample contains [Ancilla_Try1, Ancilla_Try2..., Data0, Data1...]
    # We only care about the final 7 bits for the result.
    syndrome = ((smp := GF(2)([bit_list[-7:]])) @ stean_parity.T)[0]
    err_mask = ld.decode(syndrome)
    corr = smp[0] ^ err_mask
    valid = (GF(2)([[1] * 7]) @ corr)[0] == 0
    return valid


# 2. Build the Dynamic Circuit (RUS)
def ft_stean_block(noisy=True, show=False):
    steane_gen = Circuit()
    attempt_block = build_steane_block(noisy=noisy)

    # Create a sub-circuit for the body to be repeated
    body = Circuit(attempt_block)

    steane_gen.do_while(
        body=body,
        cond=ctx.rec(
            -1
        ),  # The ancilla measurement is the last (and only) meas in the block.
        # If 1 (True), we restart.
    )

    # 3. Final Parity Check Calculation (in software/classical post-processing)
    # In the simulation, we just measure all data qubits to verify the state.
    final_meas = stim.Circuit()
    final_meas.append("M", list(range(7)))
    steane_gen.block(final_meas)
    if show:
        print_circuit(steane_gen)

    # 4. Sample and Compute Parity
    sampler = steane_gen.compile_sampler(seed=42)
    samples = sampler.sample(shots=10000)

    parities = [calculate_parity(s) for s in samples]
    zeros = parities.count(0)
    ones = parities.count(1)

    print(
        f"Collected {len(samples)} samples.",
        f"Parity 0 (Success): {zeros}",
        f"Parity 1 (Logical Error): {ones}",
        sep="\n",
    )
    return samples


samples = ft_stean_block(show=True);

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Do:                                                                                                             │
│   StimBlock:                                                                                                    │
│     R 0 1 2 3 4 5 6 7                                                                                           │
│     X_ERROR(0.001) 0 1 2 3 4 5 6 7                                                                              │
│     TICK                                                                                                        │
│     H 0 4 6                                                                                                     │
│     DEPOLARIZE1(0.001) 0 4 6                                                                                    │
│     TICK                                                                                                        │
│     CX 0 1 4 5 6 3                                                                                              │
│     DEPOLARIZE2(0.001) 0 1 4 5 6 3                                                                              │
│     TICK                                                                                                        │
│     CX 6 5 4 2 0 3                                                                                              │
│     DEPOLARIZE2(0.001) 6 5 4 2 0 3                                                                              │
│     TICK                                                                                                        │
│     CX 4 1 3 2                                                                                                  │
│     DEPOLARIZE2(0.001) 4 1 3 2                                                                                  │
│     TICK                                                                                                        │
│     CX 1 7                                                                                                      │
│     DEPOLARIZE2(0.001) 1 7                                                                                      │
│     TICK                                                                                                        │
│     CX 3 7                                                                                                      │
│     DEPOLARIZE2(0.001) 3 7                                                                                      │
│     TICK                                                                                                        │
│     CX 5 7                                                                                                      │
│     DEPOLARIZE2(0.001) 5 7                                                                                      │
│     TICK                                                                                                        │
│     M(0.001) 7                                                                                                  │
│ While rec(-1)                                                                                                   │
│ StimBlock:                                                                                                      │
│   M 0 1 2 3 4 5 6                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Collected 10000 samples.
Parity 0 (Success): 7458
Parity 1 (Logical Error): 2542